In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_excel('../database/export_binance/df_new.xlsx')
df = df.loc[df['Account'] == 'Card']
df

/var/folders/yl/s1pq6fh930j3frb4n30yfhv00000gn/T/ipykernel_54910/543722683.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.read_excel('../database/export_binance/df_new.xlsx')


,Unnamed: 0,User_ID,UTC_Time,Account,Operation,Coin,Change,Remark
7,823,11233391,2021-06-05 22:24:30,Card,Binance Card Spending,EUR,-33.000000,NaN
8,824,11233391,2021-06-07 06:24:37,Card,Card Cashback,BNB,0.000823,NaN
9,952,11233391,2021-06-08 05:10:12,Card,Card Cashback,BNB,0.003474,NaN
10,953,11233391,2021-06-08 12:31:08,Card,Binance Card Spending,EUR,-7.230000,NaN
11,954,11233391,2021-06-11 05:10:41,Card,Card Cashback,BNB,0.000743,NaN
...,...,...,...,...,...,...,...,...
7633,9308,11233391,2022-01-19 11:13:54,Card,Binance Card Spending,EUR,-28.760000,NaN
7634,9309,11233391,2022-01-19 12:17:14,Card,Binance Card Spending,EUR,-2.150000,NaN
7892,9591,11233391,2022-01-21 05:25:22,Card,Card Cashback,BNB,0.000080,NaN
7895,9594,11233391,2022-01-21 13:19:10,Card,transfer_out,EUR,-581.350000,NaN


In [9]:
df.pivot_table(index='Operation', columns='Coin', values='Change', aggfunc=np.sum)

Coin,BNB,EUR
Operation,,
Binance Card Spending,NaN,-1050.15
Card Cashback,0.119919,NaN
transfer_in,NaN,1638.77
transfer_out,NaN,-581.35
